In [1]:
import pandas as pd
import ast


def get_documents(print_info: bool = False, read_embeddings: bool = False) -> pd.DataFrame:
    docs_original = pd.read_csv("DRAGONball/en/docs.csv", usecols=["doc_id", "domain", "content"])
    docs_manipulated_single_textual = pd.read_csv(
        "additional_data/docs/textual_manipulations_result.csv",
        usecols=["doc_id", "domain", "content", "original_doc_id"],
        dtype={"original_doc_id": "Int64"},
    )
    docs_manipulated_single_textual["original_doc_id"] = docs_manipulated_single_textual["original_doc_id"].apply(
        lambda i: [i] if pd.notna(i) else []
    )
    docs_manipulated_single_textual.rename(columns={"original_doc_id": "original_doc_ids"}, inplace=True)

    docs_manipulated_single_tabular = pd.read_csv(
        "additional_data/docs/tabular_manipulations_result.csv",
        usecols=["doc_id", "domain", "content", "original_doc_ids"],
        converters={"original_doc_ids": ast.literal_eval},
    )

    docs_manipulated_multi_textual = pd.read_csv(
        "additional_data/docs/multi_textual_manipulations.csv",
        usecols=["doc_id", "domain", "content", "original_doc_id"],
        dtype={"original_doc_id": "Int64"},
    )
    docs_manipulated_multi_textual["original_doc_id"] = docs_manipulated_multi_textual["original_doc_id"].apply(
        lambda i: [i] if pd.notna(i) else []
    )
    docs_manipulated_multi_textual.rename(columns={"original_doc_id": "original_doc_ids"}, inplace=True)

    if print_info == True:
        print(f"# original docs: {len(docs_original)}")
        print(f"# manipulated textual docs: {len(docs_manipulated_single_textual)}")
        print(f"# manipulated tabular docs: {len(docs_manipulated_single_tabular)}")
        print(f"# manipulated textual multi docs: {len(docs_manipulated_multi_textual)}")
        print(
            f"= {len(docs_original) + len(docs_manipulated_single_textual) + len(docs_manipulated_single_tabular) + len(docs_manipulated_multi_textual)} documents in total"
        )

    result = pd.concat(
        [
            docs_original,
            docs_manipulated_single_textual,
            docs_manipulated_multi_textual,
            docs_manipulated_single_tabular,
        ],
        sort=False,
    )

    return result

In [4]:
import tiktoken
import numpy as np
from utils import io_helpers

encoding = tiktoken.encoding_for_model("gpt-4o")

documents = io_helpers.get_documents()

lengths = []
num_characters = []

for text in documents["content"]:
    tokens = encoding.encode(text)
    num_characters.append(len(text))
    lengths.append(len(tokens))

print()
print(f"Avegare no. of tokens per document: {np.mean(lengths):.2f}")
print(f"Avegare no. of characters per document: {np.mean(num_characters):.2f}")


Avegare no. of tokens per document: 1655.51
Avegare no. of characters per document: 8921.64
